In [33]:
import sys
import os
import pandas as pd
from pathlib import Path
import re


def replace_cname(df, oldname, newname):
    new_names = { c: c.replace(oldname, newname) for c in df.columns if oldname in c}
    return df.rename(columns=new_names)

In [17]:
grid_search = pd.DataFrame()

RES_PATH = Path(f"../res/")
files = [os.path.join(path, name) for path, subdirs, files in os.walk(RES_PATH) for name in files]
grid_files = filter(lambda x: "gridSearch" in x, files)

for f in grid_files:
    if "gridSearch" not in f:
        continue
    df = pd.read_pickle(RES_PATH / f)
    
    additional_features = f.replace(".pkl", "").split("__")[1:]
    additional_features = map(lambda x: x.split("_"), additional_features)
    
    for f, v in additional_features:
        df[f"param_{f}"] = v
        
    grid_search = pd.concat([grid_search, df], ignore_index=True)

In [30]:
best_results = grid_search.sort_values("mean_test_score", ascending=True)
best_results = best_results.groupby("param_discretized").head(5)

best_results = best_results.drop(columns=["params", "param_features", 
                                          "param_module__input_size", "param_overlapping",
                                          "param_ctxlen", "param_discretized"])
features_cols = [c for c in best_results.columns if "param" in c]
best_results = best_results[features_cols + ["mean_test_score"]]

best_results = replace_cname(best_results, "_", " ")
best_results = replace_cname(best_results, "param", "")
best_results = replace_cname(best_results, "module", "")

In [1]:
latex_str = best_results.to_latex(index=False, float_format="%.3f")
print(re.sub(' +',' ', latex_str))

NameError: name 'best_results' is not defined